In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from statsmodels.formula.api import ols

In [86]:
df=pd.read_csv("../전처리/최종 전처리완료.csv", encoding="utf-8-sig")
df1=df.copy()
df1 # 우도면, 추자면 지워야하긴 한데 어차피 선택 안 될거라 걍 냅두겠음

,일자,년,월,행정시,행정동명,사업체수,세대수,세대당 인구,남여 비율,단독주택,...,요일별배출건수비율,요일별배출량비율,요일별배출금액비율,연_요일별배출건수비율,연_요일별배출량비율,연_요일별배출금액비율,월배출건수비율,월배출량비율,월배출금액비율,분기
0,2018-01-01,2018,1,제주시,건입동,849,4324,2.27,1.08,219,...,0.144354,0.145655,0.146692,0.146655,0.147234,0.150515,0.075550,0.086198,0.086115,1
1,2018-01-01,2018,1,제주시,용담1동,896,3665,2.13,1.07,290,...,0.141912,0.143103,0.143106,0.142269,0.143542,0.143544,0.076648,0.088469,0.088512,1
2,2018-01-01,2018,1,제주시,용담2동,1418,6470,2.43,1.03,629,...,0.144280,0.144381,0.144382,0.146597,0.147322,0.147329,0.073111,0.084472,0.084519,1
3,2018-01-01,2018,1,제주시,우도면,312,1023,1.81,1.12,470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2018-01-01,2018,1,제주시,이도1동,1608,3682,2.16,1.00,164,...,0.142814,0.145538,0.145547,0.144975,0.147748,0.147758,0.075027,0.089439,0.089490,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-30,2021,6,제주시,아라동,2369,16025,2.39,0.98,660,...,0.138061,0.137128,0.137139,0.141449,0.139911,0.139891,0.085770,0.083180,0.083189,2
54907,2021-06-30,2021,6,서귀포시,안덕면,1227,6288,1.94,1.07,4272,...,0.138462,0.136020,0.135996,0.146169,0.143766,0.143638,0.090445,0.090878,0.090997,2
54908,2021-06-30,2021,6,제주시,애월읍,3083,17111,2.18,1.05,4440,...,0.138533,0.137768,0.137817,0.143750,0.143146,0.143141,0.082609,0.079777,0.079893,2
54909,2021-06-30,2021,6,제주시,연동,5211,21203,1.98,0.96,548,...,0.140246,0.139830,0.139846,0.144565,0.142464,0.142474,0.085034,0.081787,0.081853,2


In [87]:
label_df=pd.read_csv("행정동1 군집화 label5.csv")
cluster1=[]
cluster2=[]
cluster3=[]
cluster4=[]
cluster0=[]
for i in range(len(label_df)):
    if label_df.loc[i, 'label']==0:
        cluster0.append(label_df.loc[i, '행정동명'])
    if label_df.loc[i, 'label']==1:
        cluster1.append(label_df.loc[i, '행정동명'])
    if label_df.loc[i, 'label']==2:
        cluster2.append(label_df.loc[i, '행정동명'])
    if label_df.loc[i, 'label']==3:
        cluster3.append(label_df.loc[i, '행정동명'])
    if label_df.loc[i, 'label']==4:
        cluster4.append(label_df.loc[i, '행정동명'])
cluster1

['일도2동', '이도2동', '연동', '노형동']

In [88]:
cluster0

['건입동',
 '남원읍',
 '중앙동',
 '대륜동',
 '중문동',
 '대천동',
 '이도1동',
 '삼도1동',
 '삼도2동',
 '용담2동',
 '서홍동',
 '용담1동',
 '오라동']

In [126]:
cluster1

['일도2동', '이도2동', '연동', '노형동']

In [127]:
cluster2

['동홍동', '화북동', '외도동', '삼양동', '아라동']

In [128]:
cluster3

['표선면',
 '천지동',
 '정방동',
 '도두동',
 '일도1동',
 '안덕면',
 '봉개동',
 '송산동',
 '예래동',
 '영천동',
 '이호동',
 '효돈동']

In [129]:
cluster4

['성산읍', '대정읍', '애월읍']

## cluster0

In [89]:
# 설명 변수
행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
행0_df.drop([0], inplace=True)
for 행정동 in cluster0:
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
행0_df=행0_df.sort_values(['일자', '행정동명'])
행0_df=행0_df.reset_index()
행0_df.drop(['index'], axis=1, inplace=True)

# 반응 변수
행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
배출량.drop([0], inplace=True)
for 행정동 in cluster0:
    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
배출량=배출량.sort_values(['일자', '행정동명'])
배출량=배출량.reset_index()
배출량.drop(['index'], axis=1, inplace=True)

행0_df

,일자,년,월,행정시,행정동명,사업체수,세대수,세대당 인구,남여 비율,단독주택,...,요일별배출건수비율,요일별배출량비율,요일별배출금액비율,연_요일별배출건수비율,연_요일별배출량비율,연_요일별배출금액비율,월배출건수비율,월배출량비율,월배출금액비율,분기
0,2018-01-01,2018,1,제주시,건입동,849,4324,2.27,1.08,219,...,0.144354,0.145655,0.146692,0.146655,0.147234,0.150515,0.075550,0.086198,0.086115,1
1,2018-01-01,2018,1,서귀포시,남원읍,1394,8345,2.29,1.05,5689,...,0.148100,0.146502,0.146496,0.146110,0.142898,0.142886,0.073769,0.084479,0.084518,1
2,2018-01-01,2018,1,서귀포시,대륜동,964,5586,2.41,1.03,1973,...,0.147198,0.148896,0.145855,0.149547,0.151538,0.151840,0.082233,0.091919,0.089536,1
3,2018-01-01,2018,1,서귀포시,대천동,696,5770,2.28,1.03,1882,...,0.147686,0.148200,0.148222,0.150414,0.150010,0.150061,0.075489,0.086327,0.086180,1
4,2018-01-01,2018,1,제주시,삼도1동,1452,5993,2.40,0.96,485,...,0.143954,0.145411,0.145428,0.145173,0.146334,0.146338,0.076269,0.087658,0.087708,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,2021-06-30,2021,6,제주시,용담1동,896,3420,2.03,1.05,290,...,0.140835,0.141560,0.141566,0.147172,0.149033,0.149047,0.085584,0.082073,0.082058,2
16597,2021-06-30,2021,6,제주시,용담2동,1418,6463,2.26,1.03,629,...,0.139845,0.138646,0.138644,0.146163,0.146957,0.146964,0.086037,0.082275,0.082260,2
16598,2021-06-30,2021,6,제주시,이도1동,1608,3754,2.01,0.98,164,...,0.140513,0.140244,0.140244,0.146582,0.142511,0.142489,0.086368,0.084042,0.084034,2
16599,2021-06-30,2021,6,서귀포시,중문동,936,5544,2.09,1.06,1686,...,0.140639,0.139380,0.139424,0.146497,0.143217,0.143172,0.095780,0.093419,0.093464,2


In [90]:
배출량

,일자,행정동명,배출량
0,2018-01-01,건입동,1708250
1,2018-01-01,남원읍,1239600
2,2018-01-01,대륜동,1717700
3,2018-01-01,대천동,1765600
4,2018-01-01,삼도1동,2556750
...,...,...,...
16596,2021-06-30,용담1동,1105550
16597,2021-06-30,용담2동,2147800
16598,2021-06-30,이도1동,139750
16599,2021-06-30,중문동,2101200


In [91]:
행정동원핫1=pd.get_dummies(행0_df['행정동명'])
행정동원핫2=pd.get_dummies(행0_df['년'])
행정동원핫3=pd.get_dummies(행0_df['월'])
행정동원핫4=pd.get_dummies(행0_df['요일'])

행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
행0_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16601 entries, 0 to 16600
Columns: 356 entries, 일자 to 화
dtypes: float64(244), int64(76), object(1), uint8(35)
memory usage: 41.2+ MB


In [92]:
y=배출량['배출량']## 반응 변수
전체0df=pd.concat([행0_df, y], axis=1)
전체0df

,일자,사업체수,세대수,세대당 인구,남여 비율,단독주택,창고시설,제2종근린생활시설,제1종근린생활시설,공동주택,...,10,11,12,금,목,수,일,토,화,배출량
0,2018-01-01,849,4324,2.27,1.08,219,24,41,55,50,...,0,0,0,0,0,0,0,0,0,1708250
1,2018-01-01,1394,8345,2.29,1.05,5689,3846,438,482,101,...,0,0,0,0,0,0,0,0,0,1239600
2,2018-01-01,964,5586,2.41,1.03,1973,400,202,153,272,...,0,0,0,0,0,0,0,0,0,1717700
3,2018-01-01,696,5770,2.28,1.03,1882,593,145,146,185,...,0,0,0,0,0,0,0,0,0,1765600
4,2018-01-01,1452,5993,2.40,0.96,485,5,41,58,46,...,0,0,0,0,0,0,0,0,0,2556750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,2021-06-30,896,3420,2.03,1.05,290,3,36,46,37,...,0,0,0,0,0,1,0,0,0,1105550
16597,2021-06-30,1418,6463,2.26,1.03,629,21,95,64,58,...,0,0,0,0,0,1,0,0,0,2147800
16598,2021-06-30,1608,3754,2.01,0.98,164,3,36,59,25,...,0,0,0,0,0,1,0,0,0,139750
16599,2021-06-30,936,5544,2.09,1.06,1686,597,231,204,220,...,0,0,0,0,0,1,0,0,0,2101200


In [93]:
cols=list(전체0df.columns)
corr_1=전체0df.corr()
for i in range(len(corr_1)):
    for j in range(i+1, len(corr_1.iloc[0, :])):
        if corr_1.iloc[i,j]>0.8:
            print(cols[i], cols[j], corr_1.iloc[i,j])

사업체수 외국인_합계 0.9799374419102624
사업체수 총인구수_계 0.9835830158003023
사업체수 남자인구수_계 0.9735692415165836
사업체수 총인구수_0 0 14세 0.9715261570694583
사업체수 총인구수_15 0 64세 0.8702705553738265
사업체수 남자인구수_0 0 14세 0.9819113567420132
사업체수 남자인구수_15 0 64세 0.8762455725602986
사업체수 여자인구수_0 0 14세 0.9519304237647859
사업체수 여자인구수_15 0 64세 0.8648470950151945
세대수 여자인구수_계 0.8756378333034857
세대수 총인구수_65 0 100+ 0.8616455342069886
세대수 남자인구수_65 0 100+ 0.8864826772305191
세대수 여자인구수_계_비율 0.8282085654731554
세대수 총인구수_65 0 100+_비율 0.8123218539075622
세대수 남자인구수_65 0 100+_비율 0.832194523386768
세대수 일결제건수편차 0.8059385725993271
세대수 일평균1회결제금액 0.8030484821627889
세대수 월평균결제건수 0.8059137814614716
세대수 월결제건수편차 0.8614280177338405
세대수 월평균결제금액 0.8781794134064989
세대수 연평균결제건수 0.8157642315390092
세대수 연결제건수편차 0.8392370740783308
세대수 연평균결제금액 0.8877374509933061
세대수 연_요일평균결제건수 0.8116220245258502
세대수 연_요일결제건수편차 0.8149297080024984
세대수 연_요일평균결제금액 0.8518403930543803
세대수 연_월결제건수_패스트푸드 0.8299961992104252
세대수 연_월부페결제금액비율 0.8037327304798022
세대수 요일평균결제건수 0.81022291373236

여자인구수_계_비율 총인구수_65 0 100+_비율 0.9928379143280672
여자인구수_계_비율 남자인구수_65 0 100+_비율 0.992574938697954
여자인구수_계_비율 월평균결제금액 0.8005198728614265
여자인구수_계_비율 연평균결제금액 0.8066022746999715
총인구수_15 0 64세_비율 남자인구수_15 0 64세_비율 0.9940764339127641
총인구수_15 0 64세_비율 여자인구수_15 0 64세_비율 0.9971682215847691
총인구수_65 0 100+_비율 남자인구수_65 0 100+_비율 0.9709344432196729
남자인구수_15 0 64세_비율 여자인구수_15 0 64세_비율 0.9830880913941791
남자인구수_65 0 100+_비율 월평균결제금액 0.8007382086166336
남자인구수_65 0 100+_비율 연평균결제금액 0.805781135773099
여자인구수_65 0 100+_비율 확진자수 0.8691875601123662
여자인구수_65 0 100+_비율 2020 0.8634750140787438
확진자수 2020 0.9031069241461377
10만명당 발생률 일별예상승객편차 0.8090144989895387
일별예상승객비율 위도 0.9999999999998579
지점번호 평균 기온 0.9864012562347474
지점번호 최저 기온 0.9853081362545537
지점번호 경도 0.9269137575285489
지점번호 평균기온월편차 0.9252610471531454
지점번호 최고기온월편차 0.9247837229385515
평균 기온 최저 기온 0.9546253969194293
평균 기온 경도 0.9290590032090089
평균 기온 평균기온월편차 0.9303138645445042
평균 기온 최고기온월편차 0.9234992951979608
최저 기온 경도 0.904886483855051
최저 기온 평균기온월편차 0.899923708507653

연_요일평균결제건수 요일결제건수편차 0.9384152387309687
연_요일평균결제건수 요일평균결제금액 0.8802603565781107
연_요일평균결제건수 요일평균1회결제금액 0.9584572608750901
연_요일평균결제건수 월_일평균결제건수 0.9734262073719712
연_요일평균결제건수 월_일결제건수편차 0.8808482595834094
연_요일평균결제건수 월_일평균결제금액 0.8016614103223291
연_요일결제건수편차 연_요일평균결제금액 0.9427119114977538
연_요일결제건수편차 요일부페결제금액비율 0.8660156251295754
연_요일결제건수편차 연_월결제건수_마트/슈퍼마켓 0.8199175524226135
연_요일결제건수편차 연_월마트/슈퍼마켓결제금액비율 0.8751333826946219
연_요일결제건수편차 연_월결제건수_한식 0.8586690220319119
연_요일결제건수편차 연_월식품결제금액비율 0.8851782336280247
연_요일결제건수편차 연_월결제건수_패스트푸드 0.896952813109626
연_요일결제건수편차 연_월패스트푸드결제금액비율 0.8863638138103758
연_요일결제건수편차 연_월결제건수_간식 0.8847388076817728
연_요일결제건수편차 연_월간식결제금액비율 0.8914382124624285
연_요일결제건수편차 연_월결제건수_아시아음식 0.8646658610710423
연_요일결제건수편차 연_월부페결제금액비율 0.9526413955492924
연_요일결제건수편차 요일평균결제건수 0.929291622231756
연_요일결제건수편차 요일결제건수편차 0.9840049007010313
연_요일결제건수편차 요일평균결제금액 0.9419022077939895
연_요일결제건수편차 요일평균1회결제금액 0.9142932126888212
연_요일결제건수편차 월_일평균결제건수 0.8977528559529571
연_요일결제건수편차 월_일결제건수편차 0.8962105167352902
연_요일결제건수편

요일아시아음식결제금액 요일아시아음식결제건수비율 0.8087348613655977
요일농축수산물결제금액 연_월결제금액_농축수산물 0.8612361892746775
요일농축수산물결제금액비율 요일배달결제건수 0.9958047990387682
요일농축수산물결제금액비율 연_월농축수산물결제금액비율 0.8179386498692656
요일농축수산물결제금액비율 연_월결제건수_배달 0.8143381795202211
요일배달결제건수 연_월농축수산물결제금액비율 0.8193550793903733
요일배달결제건수 연_월결제건수_배달 0.8211365981120679
요일배달결제금액 요일배달결제건수비율 0.9885390832151181
요일배달결제금액 연_월농축수산물결제금액비율 0.850053923644799
요일배달결제금액 연_월결제건수_배달 0.8377080489560145
요일배달결제금액 연_월결제금액_배달 0.9514112422327659
요일배달결제금액 연_월배달결제건수비율 0.9410557613707902
요일배달결제건수비율 연_월농축수산물결제금액비율 0.8846025223806261
요일배달결제건수비율 연_월결제건수_배달 0.8765493556754786
요일배달결제건수비율 연_월결제금액_배달 0.9426650002341708
요일배달결제건수비율 연_월배달결제건수비율 0.9523273708341199
요일배달결제금액비율 요일양식결제건수 0.9612463501129672
요일양식결제금액비율 요일주점및주류판매결제건수 0.9830564213498932
요일양식결제금액비율 연_월양식결제금액비율 0.8156400707803646
요일주점및주류판매결제금액 요일주점및주류판매결제건수비율 0.9055679188817849
요일주점및주류판매결제금액비율 요일부페결제금액 0.8265839626571498
요일부페결제건수 요일부페결제건수비율 0.9345120416806552
요일부페결제금액비율 연_월결제건수_마트/슈퍼마켓 0.8368210401621371
요일부페결제금액비율 연_월마트/슈퍼마켓결제

월평균배출량 요일평균1회배출금액 0.9135517718713848
월평균배출량 연_요일평균배출건수 0.8260801654791396
월평균배출량 연_요일배출건수편차 0.9706463816881036
월평균배출량 연_요일평균배출량 0.9859275598178419
월평균배출량 연_요일배출량편차 0.9705623141509824
월평균배출량 연_요일평균배출금액 0.9043343739942378
월평균배출량 중문동 0.9745125106274564
월배출량편차 월평균배출금액 0.9016125753689587
월배출량편차 월평균1회배출금액 0.942808156113342
월배출량편차 월_일배출건수편차 0.9948776082579468
월배출량편차 월_일평균배출량 0.978406140934733
월배출량편차 월_일배출량편차 0.9937124032196818
월배출량편차 월_일평균배출금액 0.8771189854399576
월배출량편차 월_일평균1회배출금액 0.9638220180258735
월배출량편차 요일평균배출건수 0.8921915990251149
월배출량편차 요일배출건수편차 0.9911101393957635
월배출량편차 요일평균배출량 0.9783548758252746
월배출량편차 요일배출량편차 0.9910920510594804
월배출량편차 요일평균배출금액 0.9045543203251531
월배출량편차 요일평균1회배출금액 0.9395733627292562
월배출량편차 연_요일배출건수편차 0.9788382156238639
월배출량편차 연_요일평균배출량 0.969577545566867
월배출량편차 연_요일배출량편차 0.978927835591711
월배출량편차 연_요일평균배출금액 0.8853694477127518
월배출량편차 중문동 0.9758070930031922
월평균배출금액 월평균1회배출금액 0.8295352163556508
월평균배출금액 월_일배출건수편차 0.8931126647070996
월평균배출금액 월_일평균배출량 0.909645010935788
월평균배출금액 월

In [94]:
전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
"분뇨.쓰레기처리시설":'',
'건설이후(일)':'건설이후',
'건설이후(일)_std':'건설이후_std',
'내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
'내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
'내국인_행태별 0 패키지':'내국인_행태별_패키지',
'내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
'내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
'내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
'내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
'내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
'내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
'총인구수_0 0 14세':'총인구수_0_14세',
'총인구수_15 0 64세':'총인구수_15_64세',
'총인구수_65 0 100+':'총인구수_65_100',
'남자인구수_0 0 14세':'남자인구수_0_14세',
'남자인구수_15 0 64세':'남자인구수_15_64세',
'남자인구수_65 0 100+':'남자인구수_65_100',
'여자인구수_0 0 14세':'여자인구수_0_14세',
'여자인구수_15 0 64세':'여자인구수_15_64세',
'여자인구수_65 0 100+':'여자인구수_65_100',
'총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
'총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
'총인구수_65 0 100+_비율':'총인구수_65_100_비율',
'남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
'남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
'남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
'여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
'여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
'여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
'10만명당 발생률':'율_10만명당_발생률',
'평균 기온':'평균기온',
'최저 기온':'최저기온',
'최고 기온':'최고기온',
'주말/평일':'주말_평일',
'요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
'요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
'요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
'요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
'연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
'연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
'연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
'연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
2018:'연_2018년',
2019:'연_2019년',
2020:'연_2020년',
2021:'연_2021년',
1:'월_1월',
2:'월_2월',
3:'월_3월',
4:'월_4월',
5:'월_5월',
6:'월_6월',
7:'월_7월',
8:'월_8월',
9:'월_9월',
10:'월_10월',
11:'월_11월',
12:'월_12월',
}, inplace=True)
X_df=전체0df.drop(['배출량'], axis=1)
cols=list(X_df.columns)
단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

selected_vals=[]
for col in cols:
    try:
        res = ols('배출량 ~ '+col, data=전체0df).fit()
        단순회귀df.loc[0, col]=res.rsquared
        if 단순회귀df.loc[0, col]>0.5:
            selected_vals.append(col)
    except:
        print(col)
Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
for val in selected_vals:
    Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
Rsqr_df=Rsqr_df.transpose()
Rsqr_df.rename(columns={0:"Rsqr"}, inplace=True)
Rsqr_df

,Rsqr


In [95]:
단순회귀df

,사업체수,세대수,세대당인구,남여비율,단독주택,창고시설,제2종근린생활시설,제1종근린생활시설,공동주택,동식물,...,월_8월,월_9월,월_10월,월_11월,월_12월,금,목,수,토,화
0,0.082353,0.17124,0.124133,0.011965,0.01438,0.005594,0.01614,0.002398,0.011924,0.006568,...,0.022872,0.001222,0.002218,0.000901,0.000016,0.003802,0.000031,0.001708,0.000455,0.000145


In [96]:
cluster0

['건입동',
 '남원읍',
 '중앙동',
 '대륜동',
 '중문동',
 '대천동',
 '이도1동',
 '삼도1동',
 '삼도2동',
 '용담2동',
 '서홍동',
 '용담1동',
 '오라동']

## cluster1

In [99]:
# 설명 변수
행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
행0_df.drop([0], inplace=True)
for 행정동 in cluster1:
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
행0_df=행0_df.sort_values(['일자', '행정동명'])
행0_df=행0_df.reset_index()
행0_df.drop(['index'], axis=1, inplace=True)

# 반응 변수
행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
배출량.drop([0], inplace=True)
for 행정동 in cluster1:
    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
배출량=배출량.sort_values(['일자', '행정동명'])
배출량=배출량.reset_index()
배출량.drop(['index'], axis=1, inplace=True)

#행0_df

#배출량

행정동원핫1=pd.get_dummies(행0_df['행정동명'])
행정동원핫2=pd.get_dummies(행0_df['년'])
행정동원핫3=pd.get_dummies(행0_df['월'])
행정동원핫4=pd.get_dummies(행0_df['요일'])

행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
행0_df.info()

y=배출량['배출량']## 반응 변수
전체0df=pd.concat([행0_df, y], axis=1)
#전체0df


cols=list(전체0df.columns)
corr_1=전체0df.corr()
for i in range(len(corr_1)):
    for j in range(i+1, len(corr_1.iloc[0, :])):
        if corr_1.iloc[i,j]>0.8:
            print(cols[i], cols[j], corr_1.iloc[i,j])


전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
"분뇨.쓰레기처리시설":'',
'건설이후(일)':'건설이후',
'건설이후(일)_std':'건설이후_std',
'내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
'내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
'내국인_행태별 0 패키지':'내국인_행태별_패키지',
'내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
'내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
'내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
'내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
'내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
'내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
'총인구수_0 0 14세':'총인구수_0_14세',
'총인구수_15 0 64세':'총인구수_15_64세',
'총인구수_65 0 100+':'총인구수_65_100',
'남자인구수_0 0 14세':'남자인구수_0_14세',
'남자인구수_15 0 64세':'남자인구수_15_64세',
'남자인구수_65 0 100+':'남자인구수_65_100',
'여자인구수_0 0 14세':'여자인구수_0_14세',
'여자인구수_15 0 64세':'여자인구수_15_64세',
'여자인구수_65 0 100+':'여자인구수_65_100',
'총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
'총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
'총인구수_65 0 100+_비율':'총인구수_65_100_비율',
'남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
'남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
'남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
'여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
'여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
'여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
'10만명당 발생률':'율_10만명당_발생률',
'평균 기온':'평균기온',
'최저 기온':'최저기온',
'최고 기온':'최고기온',
'주말/평일':'주말_평일',
'요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
'요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
'요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
'요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
'연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
'연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
'연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
'연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
2018:'연_2018년',
2019:'연_2019년',
2020:'연_2020년',
2021:'연_2021년',
1:'월_1월',
2:'월_2월',
3:'월_3월',
4:'월_4월',
5:'월_5월',
6:'월_6월',
7:'월_7월',
8:'월_8월',
9:'월_9월',
10:'월_10월',
11:'월_11월',
12:'월_12월',
}, inplace=True)
X_df=전체0df.drop(['배출량'], axis=1)
cols=list(X_df.columns)
단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

selected_vals=[]
for col in cols:
    try:
        res = ols('배출량 ~ '+col, data=전체0df).fit()
        단순회귀df.loc[0, col]=res.rsquared
        if 단순회귀df.loc[0, col]>0.5:
            selected_vals.append(col)
    except:
        print(col)
Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
for val in selected_vals:
    Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
Rsqr_df=Rsqr_df.transpose()
Rsqr_df.rename(columns={0:"Rsqr"}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5108 entries, 0 to 5107
Columns: 347 entries, 일자 to 화
dtypes: float64(244), int64(76), object(1), uint8(26)
memory usage: 12.6+ MB
일자 사업체수 0.9211697352908109
일자 창고시설 0.9101634775066745
일자 제2종근린생활시설 0.9824767516843839
일자 제1종근린생활시설 0.9063510692414442
일자 숙박시설 0.8010694364013246
일자 초등학교수 0.9805002915766348
일자 일편차 0.8079340048971996
일자 방문편차 0.8838880193091647
일자 일표준편차 0.8430411986690598
일자 총인구수_0 0 14세 0.8240171119814352
일자 남자인구수_0 0 14세 0.8193394881054398
일자 여자인구수_0 0 14세 0.8281551092851871
일자 유흥단란주점 0.920747062287201
일자 휴게음식점 0.992764111599314
일자 음식점면적평균 0.9421135892720524
일자 일평균1회결제금액 0.8119825554173328
일자 월평균결제건수 0.8001110564061895
일자 월결제건수편차 0.8190189322327643
일자 월평균결제금액 0.8072041283541416
일자 연평균결제건수 0.8060849849336956
일자 연결제건수편차 0.8150648295503797
일자 연평균결제금액 0.8095071332114078
일자 연_월마트/슈퍼마켓결제금액비율 0.8008182907261031
일자 연_월부페결제금액비율 0.8071810717305209
일자 요일평균결제건수 0.804636939943001
사업체수 창고시설 0.8338350434732368
사업체수 제2종근린생활시설 0.880475896409

중학교수 연_월부페결제금액비율 0.9619713226671222
중학교수 요일평균결제건수 0.9811871036072133
중학교수 요일결제건수편차 0.9311508538895101
중학교수 요일평균결제금액 0.931916976053338
중학교수 요일평균1회결제금액 0.9434366451579307
중학교수 월_일평균결제건수 0.9605729392666774
중학교수 월_일결제건수편차 0.9122518843391569
중학교수 월_일평균결제금액 0.9233935449357037
중학교수 연평균배출건수 0.8746958812855026
중학교수 연평균배출량 0.8148841479978778
중학교수 요일평균배출건수 0.8534184841935687
중학교수 연_요일평균배출건수 0.8202375822167446
중학교수 분기 0.816496580927726
고등학교수 방문편차 0.949418184212619
고등학교수 외국인_합계 0.9525609049844385
고등학교수 총인구수_계 0.9530520430178347
고등학교수 남자인구수_계 0.9519807471438603
고등학교수 여자인구수_계 0.8877220473299517
고등학교수 총인구수_0 0 14세 0.9700909061101152
고등학교수 총인구수_65 0 100+ 0.8878027347592133
고등학교수 남자인구수_0 0 14세 0.9693280733023697
고등학교수 남자인구수_65 0 100+ 0.8873824269735582
고등학교수 여자인구수_0 0 14세 0.9705474901950724
고등학교수 유흥단란주점 0.9525677432074696
고등학교수 휴게음식점 0.8517876747279121
고등학교수 일반음식점 0.9176913772899922
고등학교수 음식점면적평균 0.9232852710883004
고등학교수 일 0.9349408843359311
고등학교수 일평균결제건수 0.9349810231356045
고등학교수 일결제건수편차 0.8978822745593

총인구수_계 요일평균1회결제금액 0.9260693856353978
총인구수_계 월_일평균결제건수 0.9035863697164574
총인구수_계 월_일결제건수편차 0.8832857458280207
총인구수_계 월_일평균결제금액 0.866023305393659
총인구수_계 연평균배출건수 0.8081053400373052
남자인구수_계 여자인구수_계 0.9516953504072083
남자인구수_계 총인구수_0 0 14세 0.9912929245499269
남자인구수_계 총인구수_65 0 100+ 0.9528975679359105
남자인구수_계 남자인구수_0 0 14세 0.9924304835401665
남자인구수_계 남자인구수_65 0 100+ 0.9501053216406466
남자인구수_계 여자인구수_0 0 14세 0.9899753887497316
남자인구수_계 유흥단란주점 0.9490876314457899
남자인구수_계 휴게음식점 0.8459769529506157
남자인구수_계 일반음식점 0.8371918666255694
남자인구수_계 음식점면적평균 0.8925957918310066
남자인구수_계 일 0.9249586356352993
남자인구수_계 일평균결제건수 0.9021695402390236
남자인구수_계 일결제건수편차 0.8823534124725084
남자인구수_계 일평균결제금액 0.8648184582542517
남자인구수_계 일평균1회결제금액 0.9487778986709325
남자인구수_계 월평균결제건수 0.915884082113896
남자인구수_계 월결제건수편차 0.9387311192661276
남자인구수_계 월평균결제금액 0.9135616566642424
남자인구수_계 월평균1회결제금액 0.9365591455953138
남자인구수_계 연평균결제건수 0.9232059829456782
남자인구수_계 연결제건수편차 0.9380705707641206
남자인구수_계 연평균결제금액 0.9150913451928763
남자인구수_계 연평균1회결제금액 0.91778360

여자인구수_0 0 14세_비율 일반음식점 0.8147586344384139
여자인구수_0 0 14세_비율 음식점면적평균 0.8086661904655587
여자인구수_0 0 14세_비율 연_월마트/슈퍼마켓결제건수비율 0.8118468361010028
여자인구수_15 0 64세_비율 면적당일반음식점개수 0.9455888957449232
여자인구수_15 0 64세_비율 요일식품결제금액 0.85548222462576
여자인구수_15 0 64세_비율 요일식품결제건수비율 0.842382689009652
여자인구수_15 0 64세_비율 연_월결제금액_식품 0.828722044990676
여자인구수_15 0 64세_비율 연_월식품결제건수비율 0.8346598960648902
여자인구수_15 0 64세_비율 이도2동 0.8839147553823524
여자인구수_65 0 100+_비율 확진자수 0.8691875601123874
여자인구수_65 0 100+_비율 2020 0.8634750140787619
확진자수 2020 0.9031069241461409
10만명당 발생률 일별예상승객편차 0.8090144989895339
지점번호 평균 기온 0.9864014130896585
지점번호 최저 기온 0.985971303868574
지점번호 경도 0.9284216867029321
지점번호 평균기온월편차 0.9269065003001838
지점번호 최고기온월편차 0.9263091336293424
지점번호 연_요일별배출금액비율 0.8513167764766799
평균 기온 최저 기온 0.9555728058066961
평균 기온 경도 0.9355916100819185
평균 기온 평균기온월편차 0.9367681119054352
평균 기온 최고기온월편차 0.9301221636476451
평균 기온 연_요일별배출금액비율 0.8701305507915573
최저 기온 경도 0.901703761910757
최저 기온 평균기온월편차 0.8969788068260167
최저 기온 최고기온월편차 0.9040499

일평균결제건수 요일결제건수편차 0.9240634756145532
일평균결제건수 요일평균결제금액 0.9221775820686692
일평균결제건수 요일평균1회결제금액 0.9863837021600858
일평균결제건수 월_일평균결제건수 1.0
일평균결제건수 월_일결제건수편차 0.9614312487754323
일평균결제건수 월_일평균결제금액 0.9696204558452086
일결제건수편차 일평균결제금액 0.982029656588649
일결제건수편차 일평균1회결제금액 0.9341121067703212
일결제건수편차 월평균결제건수 0.9312671314364057
일결제건수편차 월결제건수편차 0.9399984194058436
일결제건수편차 월평균결제금액 0.9355561865624523
일결제건수편차 월평균1회결제금액 0.8943789822473469
일결제건수편차 연평균결제건수 0.9224943002024846
일결제건수편차 연결제건수편차 0.9098445980537153
일결제건수편차 연평균결제금액 0.9198413185972447
일결제건수편차 연평균1회결제금액 0.8888641784856119
일결제건수편차 연_요일평균결제건수 0.9219856782545098
일결제건수편차 연_요일결제건수편차 0.8810870764295784
일결제건수편차 연_요일평균결제금액 0.8844568678133881
일결제건수편차 요일부페결제금액비율 0.9093911623448262
일결제건수편차 연_월결제건수_마트/슈퍼마켓 0.906416919397091
일결제건수편차 연_월마트/슈퍼마켓결제금액비율 0.8756981540357649
일결제건수편차 연_월결제건수_한식 0.838802869903389
일결제건수편차 연_월식품결제금액비율 0.812336113808374
일결제건수편차 연_월패스트푸드결제금액비율 0.8790369466788531
일결제건수편차 연_월결제건수_간식 0.8636012848772436
일결제건수편차 연_월간식결제금액비율 0.8703794796900703
일결제건수편차

연_요일평균결제금액 연_요일별결제금액비율 0.8386813284820883
연_요일평균결제금액 요일마트/슈퍼마켓결제건수 0.8813539764114522
연_요일평균결제금액 요일부페결제금액비율 0.9164917257330619
연_요일평균결제금액 연_월결제건수_마트/슈퍼마켓 0.9167794575972511
연_요일평균결제금액 연_월마트/슈퍼마켓결제금액비율 0.8644155384752547
연_요일평균결제금액 연_월결제건수_한식 0.8377497349108547
연_요일평균결제금액 연_월식품결제금액비율 0.8039201339645353
연_요일평균결제금액 연_월결제건수_패스트푸드 0.8014126135570824
연_요일평균결제금액 연_월패스트푸드결제금액비율 0.8713617300496876
연_요일평균결제금액 연_월결제건수_간식 0.844110268291559
연_요일평균결제금액 연_월간식결제금액비율 0.8522030777275754
연_요일평균결제금액 연_월결제건수_아시아음식 0.8667069527903939
연_요일평균결제금액 연_월부페결제금액비율 0.96691348680867
연_요일평균결제금액 요일평균결제건수 0.9649903005248641
연_요일평균결제금액 요일결제건수편차 0.9832436992667969
연_요일평균결제금액 요일평균결제금액 0.989700054663588
연_요일평균결제금액 요일평균1회결제금액 0.9044250051737464
연_요일평균결제금액 월_일평균결제건수 0.9179636874024139
연_요일평균결제금액 월_일결제건수편차 0.8844568678133881
연_요일평균결제금액 월_일평균결제금액 0.8970310066254804
연_요일결제금액편차 요일결제금액편차 0.8536947628637189
연_요일평균1회결제금액 월_일별결제건수비율 0.8642208030021213
월_일별결제금액비율 연_월결제건수비율 0.893605407006731
연_월결제금액비율 연_요일별결제건수비율 0.9750331241833762
연_월

요일결제건수편차 월_일평균결제금액 0.9013528365090006
요일평균결제금액 요일평균1회결제금액 0.9098058538171765
요일평균결제금액 월_일평균결제건수 0.9221775820686692
요일평균결제금액 월_일결제건수편차 0.8892693421135289
요일평균결제금액 월_일평균결제금액 0.9014078189727093
요일평균1회결제금액 월_일평균결제건수 0.9863837021600858
요일평균1회결제금액 월_일결제건수편차 0.975273448194944
요일평균1회결제금액 월_일평균결제금액 0.9601225517865826
월_일평균결제건수 월_일결제건수편차 0.9614312487754323
월_일평균결제건수 월_일평균결제금액 0.9696204558452086
월_일결제건수편차 월_일평균결제금액 0.982029656588649
일.1 연배출건수편차 0.8950963172894949
일.1 연배출량편차 0.8974389143361894
연평균배출건수 연평균배출량 0.9412215005605893
연평균배출건수 월평균배출건수 0.8487147571635907
연평균배출건수 월평균배출량 0.8552713924602161
연평균배출건수 요일평균배출건수 0.9293495952442947
연평균배출건수 요일평균배출량 0.8347193099631562
연평균배출건수 연_요일평균배출건수 0.9350698748303086
연평균배출건수 연_요일평균배출량 0.8190867545189889
연평균배출건수 분기 0.9522722479567867
연배출건수편차 연배출량편차 0.9994766216333055
연평균배출량 월평균배출건수 0.8462659922296805
연평균배출량 월평균배출량 0.8647677138340657
연평균배출량 요일평균배출건수 0.9128595630937549
연평균배출량 요일평균배출량 0.8369474571547005
연평균배출량 연_요일평균배출건수 0.8842780691096139
연평균배출량 연_요일평균배출량 0.86817782

In [100]:
Rsqr_df

,Rsqr
교육연구시설,0.500215
건설이후,0.518968
거주편차,0.501348
총인구수_계,0.506301
남자인구수_계,0.507865
여자인구수_계,0.504687
총인구수_15_64세,0.505320
남자인구수_15_64세,0.506421
여자인구수_0_14세,0.500020
여자인구수_15_64세,0.504033


In [101]:
단순회귀df

,사업체수,세대수,세대당인구,남여비율,단독주택,창고시설,제2종근린생활시설,제1종근린생활시설,공동주택,동식물,...,월_8월,월_9월,월_10월,월_11월,월_12월,금,목,수,토,화
0,0.32279,0.363606,0.018737,0.010362,0.063986,0.341599,0.391481,0.338051,0.419706,0.236232,...,0.028255,0.000112,0.011331,0.003957,7.234805e-07,0.01436,0.000432,0.00361,0.004957,0.000311


## cluster2

In [102]:
# 설명 변수
행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
행0_df.drop([0], inplace=True)
for 행정동 in cluster2:
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
행0_df=행0_df.sort_values(['일자', '행정동명'])
행0_df=행0_df.reset_index()
행0_df.drop(['index'], axis=1, inplace=True)

# 반응 변수
행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
배출량.drop([0], inplace=True)
for 행정동 in cluster2:
    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
배출량=배출량.sort_values(['일자', '행정동명'])
배출량=배출량.reset_index()
배출량.drop(['index'], axis=1, inplace=True)

#행0_df

#배출량

행정동원핫1=pd.get_dummies(행0_df['행정동명'])
행정동원핫2=pd.get_dummies(행0_df['년'])
행정동원핫3=pd.get_dummies(행0_df['월'])
행정동원핫4=pd.get_dummies(행0_df['요일'])

행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
행0_df.info()

y=배출량['배출량']## 반응 변수
전체0df=pd.concat([행0_df, y], axis=1)
#전체0df


cols=list(전체0df.columns)
corr_1=전체0df.corr()
for i in range(len(corr_1)):
    for j in range(i+1, len(corr_1.iloc[0, :])):
        if corr_1.iloc[i,j]>0.8:
            print(cols[i], cols[j], corr_1.iloc[i,j])


전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
"분뇨.쓰레기처리시설":'',
'건설이후(일)':'건설이후',
'건설이후(일)_std':'건설이후_std',
'내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
'내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
'내국인_행태별 0 패키지':'내국인_행태별_패키지',
'내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
'내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
'내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
'내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
'내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
'내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
'총인구수_0 0 14세':'총인구수_0_14세',
'총인구수_15 0 64세':'총인구수_15_64세',
'총인구수_65 0 100+':'총인구수_65_100',
'남자인구수_0 0 14세':'남자인구수_0_14세',
'남자인구수_15 0 64세':'남자인구수_15_64세',
'남자인구수_65 0 100+':'남자인구수_65_100',
'여자인구수_0 0 14세':'여자인구수_0_14세',
'여자인구수_15 0 64세':'여자인구수_15_64세',
'여자인구수_65 0 100+':'여자인구수_65_100',
'총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
'총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
'총인구수_65 0 100+_비율':'총인구수_65_100_비율',
'남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
'남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
'남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
'여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
'여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
'여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
'10만명당 발생률':'율_10만명당_발생률',
'평균 기온':'평균기온',
'최저 기온':'최저기온',
'최고 기온':'최고기온',
'주말/평일':'주말_평일',
'요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
'요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
'요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
'요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
'연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
'연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
'연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
'연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
2018:'연_2018년',
2019:'연_2019년',
2020:'연_2020년',
2021:'연_2021년',
1:'월_1월',
2:'월_2월',
3:'월_3월',
4:'월_4월',
5:'월_5월',
6:'월_6월',
7:'월_7월',
8:'월_8월',
9:'월_9월',
10:'월_10월',
11:'월_11월',
12:'월_12월',
}, inplace=True)
X_df=전체0df.drop(['배출량'], axis=1)
cols=list(X_df.columns)
단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

selected_vals=[]
for col in cols:
    try:
        res = ols('배출량 ~ '+col, data=전체0df).fit()
        단순회귀df.loc[0, col]=res.rsquared
        if 단순회귀df.loc[0, col]>0.5:
            selected_vals.append(col)
    except:
        print(col)
Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
for val in selected_vals:
    Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
Rsqr_df=Rsqr_df.transpose()
Rsqr_df.rename(columns={0:"Rsqr"}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6385 entries, 0 to 6384
Columns: 348 entries, 일자 to 화
dtypes: float64(244), int64(76), object(1), uint8(27)
memory usage: 15.8+ MB
일자 숙박시설 0.8867633154609837
일자 초등학교수 0.9244485391617363
일자 중학교수 0.9409021026443064
일자 일표준편차 0.801947196505706
일자 장기2방문표준편차 0.8250934301448049
일자 장기3거주편차 0.8102836000273735
일자 장기3방문표준편차 0.8621467474405133
일자 장기4거주편차 0.8502744091841329
일자 총인구수_0 0 14세_비율 0.8975440239273309
일자 여자인구수_0 0 14세_비율 0.9607187241816574
일자 월평균결제금액 0.8469197833881079
일자 연평균결제금액 0.8581629979540055
사업체수 공동주택 0.9233634821641463
사업체수 숙박시설 0.9052264351314511
사업체수 초등학교수 0.8712515243244247
사업체수 중학교수 0.8471277831758848
사업체수 고등학교수 0.9227398883212152
사업체수 방문편차 0.934001921724069
사업체수 방문표준편차 0.8386274329848846
사업체수 장기1거주편차 0.8562674645275301
사업체수 장기2방문표준편차 0.8574665746411746
사업체수 장기3거주편차 0.8661210830402849
사업체수 장기3방문표준편차 0.9300555857431978
사업체수 장기4거주편차 0.9270079729014323
사업체수 장기4일표준편차 0.844899650478571
사업체수 장기4방문편차 0.8344650782591672
사업체수 내국인_합계 0.9

분뇨.쓰레기처리시설 건설이후(일) 0.9028214948120371
분뇨.쓰레기처리시설 총인구수_15 0 64세_비율 0.9625128993029438
분뇨.쓰레기처리시설 남자인구수_15 0 64세_비율 0.9454085141230223
분뇨.쓰레기처리시설 여자인구수_15 0 64세_비율 0.9659717849703352
분뇨.쓰레기처리시설 일.1 0.8343088609719975
분뇨.쓰레기처리시설 월_일평균1회배출금액 0.8231373454292124
분뇨.쓰레기처리시설 요일평균1회배출금액 0.8125463740859068
건설이후(일) 총인구수_15 0 64세_비율 0.8976770095749647
건설이후(일) 남자인구수_15 0 64세_비율 0.8899821263101811
건설이후(일) 여자인구수_15 0 64세_비율 0.8956444906332357
건설이후(일)_std 외도동 0.9185586535437353
초등학교수 중학교수 0.9468419827298833
초등학교수 고등학교수 0.81052328860173
초등학교수 방문편차 0.8711890761300256
초등학교수 방문표준편차 0.819090032309248
초등학교수 장기1거주편차 0.8248078216202143
초등학교수 장기2방문표준편차 0.8445843750046167
초등학교수 장기3거주편차 0.8351452960697278
초등학교수 장기3방문표준편차 0.899873892037449
초등학교수 장기4거주편차 0.8903512396033139
초등학교수 장기4일표준편차 0.8361184054841537
초등학교수 장기4방문편차 0.8315974134607792
초등학교수 내국인_합계 0.8509466713474576
초등학교수 외국인_합계 0.8580637447029986
초등학교수 총인구수_계 0.8362287519737716
초등학교수 남자인구수_계 0.8773682900070204
초등학교수 총인구수_0 0 14세 0.8707123519334207
초등학교수 총인구수_

장기4일표준편차 장기4방문편차 0.9841261555500277
장기4일표준편차 내국인_합계 0.8200321589617423
장기4일표준편차 외국인_합계 0.830186271915415
장기4일표준편차 총인구수_계 0.8201471286950005
장기4일표준편차 남자인구수_계 0.8380109838928586
장기4일표준편차 총인구수_0 0 14세 0.8436780600878444
장기4일표준편차 남자인구수_0 0 14세 0.839903095299198
장기4일표준편차 여자인구수_0 0 14세 0.8466102731516613
장기4일표준편차 유흥단란주점 0.80731503064598
장기4일표준편차 음식점면적합 0.8177058785466251
장기4일표준편차 월결제건수편차 0.8120966147169107
장기4일표준편차 연결제건수편차 0.8092977937620913
장기4일표준편차 삼양동 0.8183503300437007
장기4방문편차 내국인_합계 0.8147230568888548
장기4방문편차 외국인_합계 0.8213163772007942
장기4방문편차 총인구수_계 0.8112978497984435
장기4방문편차 남자인구수_계 0.829142305566696
장기4방문편차 총인구수_0 0 14세 0.8342013883414652
장기4방문편차 남자인구수_0 0 14세 0.8303957961408013
장기4방문편차 여자인구수_0 0 14세 0.8371729587466332
장기4방문편차 유흥단란주점 0.8019205115507744
장기4방문편차 음식점면적합 0.8123362924303975
장기4방문편차 월결제건수편차 0.8037457770795119
장기4방문편차 삼양동 0.8124450965628114
장기4방문표준편차 단기1방문편차 0.9819878332753337
단기1방문표준편차 단기2방문편차 0.9773442545475789
단기2방문표준편차 단기3방문편차 0.9696242730025308
단기3방문표준편차 단기4방문편차 0.98823

남자인구수_계_비율 일별예상승객비율 0.8418386936102917
남자인구수_계_비율 위도 0.8418386936102948
남자인구수_계_비율 월결제금액편차 0.8013148013149642
남자인구수_계_비율 연결제금액편차 0.8448339414542061
남자인구수_계_비율 분기 0.8418386936103416
여자인구수_계_비율 총인구수_65 0 100+_비율 0.9861357353168063
여자인구수_계_비율 남자인구수_65 0 100+_비율 0.9795891205035734
총인구수_0 0 14세_비율 남자인구수_0 0 14세_비율 0.8815620401870637
총인구수_0 0 14세_비율 여자인구수_0 0 14세_비율 0.9334913224178164
총인구수_0 0 14세_비율 휴게음식점 0.8649170639332382
총인구수_0 0 14세_비율 음식점면적평균 0.8365286518171463
총인구수_15 0 64세_비율 남자인구수_15 0 64세_비율 0.9925844321114043
총인구수_15 0 64세_비율 여자인구수_15 0 64세_비율 0.9969993686428948
총인구수_15 0 64세_비율 일.1 0.8442130368368711
총인구수_15 0 64세_비율 월_일평균1회배출금액 0.8305962153807886
총인구수_15 0 64세_비율 요일평균1회배출금액 0.8221632346902467
총인구수_65 0 100+_비율 남자인구수_65 0 100+_비율 0.9326520445266232
남자인구수_0 0 14세_비율 휴게음식점 0.8764385305550734
남자인구수_15 0 64세_비율 여자인구수_15 0 64세_비율 0.9801963503622598
여자인구수_0 0 14세_비율 월평균결제금액 0.8011231811213352
여자인구수_0 0 14세_비율 연평균결제금액 0.8244139580301819
여자인구수_15 0 64세_비율 일.1 0.8669244189779203
여자인구수_15 

일결제금액편차 월결제금액편차 0.8015231912268769
일결제금액편차 월_일결제금액편차 1.0
일평균1회결제금액 월평균결제건수 0.9474818427889121
일평균1회결제금액 월결제건수편차 0.9436117435242927
일평균1회결제금액 월평균1회결제금액 0.9002157084748151
일평균1회결제금액 연평균결제건수 0.8970989211554941
일평균1회결제금액 연결제건수편차 0.8502870053110254
일평균1회결제금액 연평균1회결제금액 0.8671564444945222
일평균1회결제금액 연_요일평균결제건수 0.8696570413479284
일평균1회결제금액 요일간식결제건수 0.8077085979003109
일평균1회결제금액 연_월마트/슈퍼마켓결제금액비율 0.942718895753993
일평균1회결제금액 연_월결제건수_한식 0.919180351746095
일평균1회결제금액 연_월한식결제금액비율 0.8511837435554797
일평균1회결제금액 연_월식품결제금액비율 0.9214607236928024
일평균1회결제금액 연_월결제건수_패스트푸드 0.8899883406870637
일평균1회결제금액 연_월패스트푸드결제금액비율 0.9577281616921861
일평균1회결제금액 연_월결제건수_간식 0.9372888826546303
일평균1회결제금액 연_월간식결제금액비율 0.9371201726333226
일평균1회결제금액 연_월결제건수_아시아음식 0.9011321301020474
일평균1회결제금액 연_월배달결제금액비율 0.9256308235550301
일평균1회결제금액 연_월결제건수_양식 0.8954319370177972
일평균1회결제금액 연_월부페결제금액비율 0.936889371817536
일평균1회결제금액 요일평균결제건수 0.8968042953782549
일평균1회결제금액 요일결제건수편차 0.8186770769315381
일평균1회결제금액 요일평균1회결제금액 0.957000727102638
일평균1회결제금액 월_일평균결제건수 0.9084

요일마트/슈퍼마켓결제건수비율 연_월결제금액_마트/슈퍼마켓 0.8570977336377291
요일마트/슈퍼마켓결제건수비율 연_월마트/슈퍼마켓결제건수비율 0.9330219512784195
요일마트/슈퍼마켓결제건수비율 요일결제금액편차 0.8200575114041865
요일마트/슈퍼마켓결제건수비율 연배출건수편차 0.8121268597734904
요일마트/슈퍼마켓결제건수비율 연배출량편차 0.810045243551547
요일마트/슈퍼마켓결제건수비율 연배출금액편차 0.8668258472210538
요일마트/슈퍼마켓결제건수비율 요일배출건수편차 0.8398342721638613
요일마트/슈퍼마켓결제건수비율 요일배출량편차 0.8407962015393058
요일마트/슈퍼마켓결제건수비율 요일배출금액편차 0.889892538497846
요일마트/슈퍼마켓결제건수비율 연_요일배출건수편차 0.8432499310293129
요일마트/슈퍼마켓결제건수비율 연_요일배출량편차 0.8390926946150202
요일마트/슈퍼마켓결제건수비율 연_요일배출금액편차 0.8600061295911041
요일마트/슈퍼마켓결제건수비율 분기 0.8813509553348672
요일마트/슈퍼마켓결제금액비율 요일한식결제건수 0.9743731076506266
요일마트/슈퍼마켓결제금액비율 요일한식결제금액 0.807147128617228
요일마트/슈퍼마켓결제금액비율 요일한식결제금액비율 0.9143146514125011
요일마트/슈퍼마켓결제금액비율 요일식품결제건수 0.8464497513498728
요일마트/슈퍼마켓결제금액비율 요일식품결제금액비율 0.9121888613780491
요일마트/슈퍼마켓결제금액비율 요일패스트푸드결제건수 0.8997615223935437
요일마트/슈퍼마켓결제금액비율 요일패스트푸드결제금액비율 0.9824124511278912
요일마트/슈퍼마켓결제금액비율 요일간식결제건수 0.9791701864777159
요일마트/슈퍼마켓결제금액비율 요일간식결제금액비율 0.9279544340477472
요일마트/슈퍼마켓결제금

일.1 연배출량편차 0.9718652530122779
일.1 연평균1회배출금액 0.8998452034045566
일.1 월배출건수편차 0.9392834708997859
일.1 월평균배출량 0.8169830114057307
일.1 월배출량편차 0.9406426844794283
일.1 월평균1회배출금액 0.8716352089010407
일.1 월_일배출건수편차 0.9134391524826959
일.1 월_일배출량편차 0.897166969268396
일.1 월_일평균1회배출금액 0.9690218199979405
일.1 요일배출건수편차 0.9503876805722904
일.1 요일평균배출량 0.8283845776121903
일.1 요일배출량편차 0.950025071967327
일.1 요일평균1회배출금액 0.9737750106319037
일.1 연_요일배출건수편차 0.9514280901523379
일.1 연_요일평균배출량 0.821034132902077
일.1 연_요일배출량편차 0.9487617746120631
일.1 분기 0.8611203847971413
연평균배출건수 연평균배출량 0.8827531407450675
연평균배출건수 월평균배출건수 0.9102868502374174
연평균배출건수 월평균배출량 0.8269098836155364
연평균배출건수 월_일평균배출건수 0.8839800345711951
연평균배출건수 요일평균배출건수 0.9664115011973957
연평균배출건수 요일평균배출량 0.8331567420683901
연평균배출건수 연_요일평균배출건수 0.97665879253827
연평균배출건수 연_요일평균배출량 0.8374118740932344
연배출건수편차 연배출량편차 0.9994779344848899
연배출건수편차 연평균1회배출금액 0.8809277683321439
연배출건수편차 월배출건수편차 0.9629798941722576
연배출건수편차 월배출량편차 0.9639396918952585
연배출건수편차 월평균1회배출금액 0.853242648220392
연배

In [104]:
Rsqr_df

,Rsqr


## cluster3

In [105]:
# 설명 변수
행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
행0_df.drop([0], inplace=True)
for 행정동 in cluster3:
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
행0_df=행0_df.sort_values(['일자', '행정동명'])
행0_df=행0_df.reset_index()
행0_df.drop(['index'], axis=1, inplace=True)

# 반응 변수
행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
배출량.drop([0], inplace=True)
for 행정동 in cluster3:
    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
배출량=배출량.sort_values(['일자', '행정동명'])
배출량=배출량.reset_index()
배출량.drop(['index'], axis=1, inplace=True)

#행0_df

#배출량

행정동원핫1=pd.get_dummies(행0_df['행정동명'])
행정동원핫2=pd.get_dummies(행0_df['년'])
행정동원핫3=pd.get_dummies(행0_df['월'])
행정동원핫4=pd.get_dummies(행0_df['요일'])

행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
행0_df.info()

y=배출량['배출량']## 반응 변수
전체0df=pd.concat([행0_df, y], axis=1)
#전체0df


cols=list(전체0df.columns)
corr_1=전체0df.corr()
for i in range(len(corr_1)):
    for j in range(i+1, len(corr_1.iloc[0, :])):
        if corr_1.iloc[i,j]>0.8:
            print(cols[i], cols[j], corr_1.iloc[i,j])


전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
"분뇨.쓰레기처리시설":'',
'건설이후(일)':'건설이후',
'건설이후(일)_std':'건설이후_std',
'내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
'내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
'내국인_행태별 0 패키지':'내국인_행태별_패키지',
'내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
'내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
'내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
'내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
'내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
'내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
'총인구수_0 0 14세':'총인구수_0_14세',
'총인구수_15 0 64세':'총인구수_15_64세',
'총인구수_65 0 100+':'총인구수_65_100',
'남자인구수_0 0 14세':'남자인구수_0_14세',
'남자인구수_15 0 64세':'남자인구수_15_64세',
'남자인구수_65 0 100+':'남자인구수_65_100',
'여자인구수_0 0 14세':'여자인구수_0_14세',
'여자인구수_15 0 64세':'여자인구수_15_64세',
'여자인구수_65 0 100+':'여자인구수_65_100',
'총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
'총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
'총인구수_65 0 100+_비율':'총인구수_65_100_비율',
'남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
'남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
'남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
'여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
'여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
'여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
'10만명당 발생률':'율_10만명당_발생률',
'평균 기온':'평균기온',
'최저 기온':'최저기온',
'최고 기온':'최고기온',
'주말/평일':'주말_평일',
'요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
'요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
'요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
'요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
'연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
'연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
'연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
'연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
2018:'연_2018년',
2019:'연_2019년',
2020:'연_2020년',
2021:'연_2021년',
1:'월_1월',
2:'월_2월',
3:'월_3월',
4:'월_4월',
5:'월_5월',
6:'월_6월',
7:'월_7월',
8:'월_8월',
9:'월_9월',
10:'월_10월',
11:'월_11월',
12:'월_12월',
}, inplace=True)
X_df=전체0df.drop(['배출량'], axis=1)
cols=list(X_df.columns)
단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

selected_vals=[]
for col in cols:
    try:
        res = ols('배출량 ~ '+col, data=전체0df).fit()
        단순회귀df.loc[0, col]=res.rsquared
        if 단순회귀df.loc[0, col]>0.5:
            selected_vals.append(col)
    except:
        print(col)
Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
for val in selected_vals:
    Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
Rsqr_df=Rsqr_df.transpose()
Rsqr_df.rename(columns={0:"Rsqr"}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15324 entries, 0 to 15323
Columns: 355 entries, 일자 to 화
dtypes: float64(244), int64(76), object(1), uint8(34)
memory usage: 38.0+ MB
사업체수 남여 비율 0.9631622769988794
사업체수 단독주택 0.9148234813927565
사업체수 창고시설 0.948950569900895
사업체수 제2종근린생활시설 0.9507753388234936
사업체수 제1종근린생활시설 0.8939733738611243
사업체수 공동주택 0.9585630450403275
사업체수 건설이후(일)_std 0.9512385277073869
사업체수 고등학교수 0.9418489052441292
사업체수 방문편차 0.945833342057148
사업체수 내국인_합계 0.8739416759578662
사업체수 외국인_합계 0.9912138496773714
사업체수 총인구수_계 0.9910501540930702
사업체수 남자인구수_계 0.9909574278964053
사업체수 여자인구수_계 0.9460751942556854
사업체수 총인구수_0 0 14세 0.9896900985842374
사업체수 총인구수_15 0 64세 0.9781206089375805
사업체수 총인구수_65 0 100+ 0.9437282749176411
사업체수 남자인구수_0 0 14세 0.9906254771346026
사업체수 남자인구수_15 0 64세 0.9695837105794035
사업체수 남자인구수_65 0 100+ 0.9443840332522838
사업체수 여자인구수_0 0 14세 0.9873027048705842
사업체수 여자인구수_15 0 64세 0.9818593753366843
사업체수 음식점면적합 0.9325537942636454
사업체수 일 0.9061690800344746
사업체수 일평균결제건수 0.86

교육연구시설 연_월결제건수_양식 0.8003523233482022
교육연구시설 송산동 0.9868665964029892
분뇨.쓰레기처리시설 연배출건수편차 0.8171303413339918
분뇨.쓰레기처리시설 연배출량편차 0.8100298226934118
분뇨.쓰레기처리시설 월배출건수편차 0.8368292425621349
분뇨.쓰레기처리시설 월배출량편차 0.8106453247748976
분뇨.쓰레기처리시설 월_일배출건수편차 0.8258387009610038
분뇨.쓰레기처리시설 요일배출건수편차 0.8448771909316035
분뇨.쓰레기처리시설 요일배출량편차 0.8370094523770836
분뇨.쓰레기처리시설 연_요일배출건수편차 0.8152288983745823
분뇨.쓰레기처리시설 연_요일배출량편차 0.8035537629500239
건설이후(일)_std 고등학교수 0.8825646811594063
건설이후(일)_std 방문편차 0.8901214218535434
건설이후(일)_std 내국인_합계 0.840032116206204
건설이후(일)_std 외국인_합계 0.9475923911038007
건설이후(일)_std 총인구수_계 0.9507221449898702
건설이후(일)_std 남자인구수_계 0.9438730697056869
건설이후(일)_std 여자인구수_계 0.9141339821928035
건설이후(일)_std 총인구수_0 0 14세 0.944142975403833
건설이후(일)_std 총인구수_15 0 64세 0.9361240043770195
건설이후(일)_std 총인구수_65 0 100+ 0.9111126911294
건설이후(일)_std 남자인구수_0 0 14세 0.9491013609762043
건설이후(일)_std 남자인구수_15 0 64세 0.9311311855267591
건설이후(일)_std 남자인구수_65 0 100+ 0.9133517338193115
건설이후(일)_std 여자인구수_0 0 14세 0.9370523909446318
건설이후(일)

장기2거주표준편차 송산동 0.8010679040596005
장기2일표준편차 장기2방문편차 0.9720003616664133
장기2방문표준편차 장기3거주편차 0.9906108235152083
장기2방문표준편차 장기3방문표준편차 0.8922229891593291
장기2방문표준편차 장기4거주편차 0.8911565428096739
장기2방문표준편차 장기4거주표준편차 0.8255082385570774
장기2방문표준편차 장기4일편차 0.8050881452086139
장기2방문표준편차 송산동 0.8670635829712703
장기3거주편차 장기3방문표준편차 0.8989812939211836
장기3거주편차 장기4거주편차 0.8989724250335296
장기3거주편차 장기4거주표준편차 0.8315679385845689
장기3거주편차 장기4일편차 0.8116295394162425
장기3거주편차 송산동 0.8707494972991071
장기3거주표준편차 장기3일편차 0.9696566108672496
장기3일표준편차 장기3방문편차 0.9684747396430838
장기3방문표준편차 장기4거주편차 0.9975343480934393
장기3방문표준편차 장기4거주표준편차 0.9010782069302797
장기3방문표준편차 장기4일편차 0.8850867423922723
장기3방문표준편차 장기4일표준편차 0.8659343890008332
장기3방문표준편차 장기4방문편차 0.8532739729285124
장기3방문표준편차 일 0.8041339235460836
장기3방문표준편차 일평균1회결제금액 0.8207490217258481
장기3방문표준편차 월결제건수편차 0.8253797255715031
장기3방문표준편차 월평균1회결제금액 0.8242332992548312
장기3방문표준편차 연결제건수편차 0.8324986167464132
장기3방문표준편차 연평균1회결제금액 0.8215764446250923
장기3방문표준편차 연_요일결제건수편차 0.8242507455674851
장기3방문표준편차 요일패스트

남자인구수_15 0 64세 연_요일결제건수편차 0.9121783098551811
남자인구수_15 0 64세 연_요일평균결제금액 0.8565415664040833
남자인구수_15 0 64세 요일한식결제금액비율 0.8355127547652688
남자인구수_15 0 64세 연_월결제건수_마트/슈퍼마켓 0.8146716719789946
남자인구수_15 0 64세 연_월한식결제금액비율 0.8783346027772087
남자인구수_15 0 64세 연_월식품결제금액비율 0.8300750217708067
남자인구수_15 0 64세 연_월결제건수_패스트푸드 0.8209157061808036
남자인구수_15 0 64세 연_월간식결제금액비율 0.8039094665309147
남자인구수_15 0 64세 연_월부페결제금액비율 0.8821023830879312
남자인구수_15 0 64세 요일평균결제건수 0.8596535617538728
남자인구수_15 0 64세 요일결제건수편차 0.9188027610914383
남자인구수_15 0 64세 요일평균결제금액 0.8904207523524158
남자인구수_15 0 64세 요일평균1회결제금액 0.8571119721664084
남자인구수_15 0 64세 월_일평균결제건수 0.8271250221272742
남자인구수_15 0 64세 월_일결제건수편차 0.8320834549004074
남자인구수_65 0 100+ 여자인구수_0 0 14세 0.9729649018367676
남자인구수_65 0 100+ 여자인구수_15 0 64세 0.9072190665262705
남자인구수_65 0 100+ 음식점면적합 0.8983337607030331
남자인구수_65 0 100+ 일 0.9099472833392142
남자인구수_65 0 100+ 일평균결제건수 0.8833609377412435
남자인구수_65 0 100+ 일결제건수편차 0.8286686010521139
남자인구수_65 0 100+ 일평균1회결제금액 0.9300350604127302
남자인구수_65 0 1

연_요일평균결제건수 연_월간식결제금액비율 0.8747686057324947
연_요일평균결제건수 연_월결제건수_아시아음식 0.8701653124009123
연_요일평균결제건수 연_월배달결제금액비율 0.8084821146542243
연_요일평균결제건수 연_월부페결제금액비율 0.9658149323634091
연_요일평균결제건수 요일평균결제건수 0.9808837592636991
연_요일평균결제건수 요일결제건수편차 0.9437465154319713
연_요일평균결제건수 요일평균결제금액 0.9050427462722944
연_요일평균결제건수 요일평균1회결제금액 0.9370961012427479
연_요일평균결제건수 월_일평균결제건수 0.9428700021124721
연_요일평균결제건수 월_일결제건수편차 0.8532777100729286
연_요일결제건수편차 연_요일평균결제금액 0.9415793868487667
연_요일결제건수편차 요일한식결제금액비율 0.86528017239701
연_요일결제건수편차 요일식품결제건수 0.8079386664543995
연_요일결제건수편차 요일식품결제금액비율 0.821135675939725
연_요일결제건수편차 요일패스트푸드결제건수 0.826439890840475
연_요일결제건수편차 요일패스트푸드결제금액비율 0.8104155429658787
연_요일결제건수편차 요일부페결제금액비율 0.8357180886950517
연_요일결제건수편차 연_월결제건수_마트/슈퍼마켓 0.833002457687358
연_요일결제건수편차 연_월마트/슈퍼마켓결제금액비율 0.8888349569039379
연_요일결제건수편차 연_월결제건수_한식 0.8941128590239463
연_요일결제건수편차 연_월한식결제금액비율 0.9148656796253176
연_요일결제건수편차 연_월결제건수_식품 0.850945933178788
연_요일결제건수편차 연_월식품결제금액비율 0.9229646350255765
연_요일결제건수편차 연_월결제건수_패스트푸드 0.9201989902697427
연_요일결제

요일배달결제금액 연_월배달결제건수비율 0.9152171134386814
요일배달결제건수비율 연_월결제건수_배달 0.8045836305240226
요일배달결제건수비율 연_월결제금액_배달 0.9130185745065725
요일배달결제건수비율 연_월배달결제건수비율 0.9267338486228235
요일배달결제금액비율 요일양식결제건수 0.9772639232466219
요일배달결제금액비율 연_월마트/슈퍼마켓결제금액비율 0.838291064855806
요일배달결제금액비율 연_월결제건수_한식 0.8219313550850259
요일배달결제금액비율 연_월식품결제금액비율 0.8436677412766158
요일배달결제금액비율 연_월결제건수_패스트푸드 0.8418189575538518
요일배달결제금액비율 연_월패스트푸드결제금액비율 0.8629111557597053
요일배달결제금액비율 연_월간식결제금액비율 0.8158131158725125
요일배달결제금액비율 연_월결제건수_아시아음식 0.8050760997879312
요일배달결제금액비율 연_월배달결제금액비율 0.8419213267738389
요일배달결제금액비율 연_월결제건수_양식 0.8202118435971928
요일배달결제금액비율 연_월부페결제금액비율 0.8176017806729087
요일배달결제금액비율 요일결제건수편차 0.802101076723712
요일배달결제금액비율 송산동 0.8212536476065454
요일양식결제건수 연_월마트/슈퍼마켓결제금액비율 0.8142744830062888
요일양식결제건수 연_월식품결제금액비율 0.8137877823477228
요일양식결제건수 연_월결제건수_패스트푸드 0.8150352242476172
요일양식결제건수 연_월패스트푸드결제금액비율 0.8356696024103222
요일양식결제건수 연_월배달결제금액비율 0.8163879492803884
요일양식결제건수 연_월결제건수_양식 0.8037075797168759
요일양식결제건수 송산동 0.8022774572176259
요일양식결제금액 요일양식결제

요일평균결제금액 요일평균1회결제금액 0.8864606817685311
요일평균결제금액 월_일평균결제건수 0.8808834114979845
요일평균결제금액 월_일결제건수편차 0.8604161331626872
요일평균1회결제금액 월_일평균결제건수 0.9728158530579921
요일평균1회결제금액 월_일결제건수편차 0.921713192834314
월_일평균결제건수 월_일결제건수편차 0.890978771586353
월_일결제건수편차 월_일평균결제금액 0.9095297822250284
일.1 연평균배출건수 0.8576674513704035
일.1 연배출건수편차 0.9735953161609999
일.1 연평균배출량 0.8457261759215964
일.1 연배출량편차 0.9579440496831384
일.1 연평균1회배출금액 0.9246761352778612
일.1 월평균배출건수 0.8026855393688817
일.1 월배출건수편차 0.9238621017510901
일.1 월평균배출량 0.8202984972357344
일.1 월배출량편차 0.8937611171267548
일.1 월평균1회배출금액 0.9138106717806663
일.1 월_일배출건수편차 0.911603659965936
일.1 월_일평균1회배출금액 0.9498018591428342
일.1 요일평균배출건수 0.8210573862137032
일.1 요일배출건수편차 0.9346118774778311
일.1 요일평균배출량 0.8403094779953042
일.1 요일배출량편차 0.9239806344318854
일.1 요일평균1회배출금액 0.9978922524564176
일.1 연_요일평균배출건수 0.8556651318078222
일.1 연_요일배출건수편차 0.9712106768186133
일.1 연_요일평균배출량 0.8419215369746035
일.1 연_요일배출량편차 0.9500403501903736
연평균배출건수 연배출건수편차 0.8241527257424065
연평균배출건수 연평균배출량 0.859612

In [106]:
Rsqr_df

,Rsqr


## cluster4

In [107]:
# 설명 변수
행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
행0_df.drop([0], inplace=True)
for 행정동 in cluster4:
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
행0_df=행0_df.sort_values(['일자', '행정동명'])
행0_df=행0_df.reset_index()
행0_df.drop(['index'], axis=1, inplace=True)

# 반응 변수
행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
배출량.drop([0], inplace=True)
for 행정동 in cluster4:
    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
배출량=배출량.sort_values(['일자', '행정동명'])
배출량=배출량.reset_index()
배출량.drop(['index'], axis=1, inplace=True)

#행0_df

#배출량

행정동원핫1=pd.get_dummies(행0_df['행정동명'])
행정동원핫2=pd.get_dummies(행0_df['년'])
행정동원핫3=pd.get_dummies(행0_df['월'])
행정동원핫4=pd.get_dummies(행0_df['요일'])

행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
행0_df.info()

y=배출량['배출량']## 반응 변수
전체0df=pd.concat([행0_df, y], axis=1)
#전체0df


cols=list(전체0df.columns)
corr_1=전체0df.corr()
for i in range(len(corr_1)):
    for j in range(i+1, len(corr_1.iloc[0, :])):
        if corr_1.iloc[i,j]>0.8:
            print(cols[i], cols[j], corr_1.iloc[i,j])


전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
"분뇨.쓰레기처리시설":'',
'건설이후(일)':'건설이후',
'건설이후(일)_std':'건설이후_std',
'내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
'내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
'내국인_행태별 0 패키지':'내국인_행태별_패키지',
'내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
'내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
'내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
'내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
'내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
'내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
'총인구수_0 0 14세':'총인구수_0_14세',
'총인구수_15 0 64세':'총인구수_15_64세',
'총인구수_65 0 100+':'총인구수_65_100',
'남자인구수_0 0 14세':'남자인구수_0_14세',
'남자인구수_15 0 64세':'남자인구수_15_64세',
'남자인구수_65 0 100+':'남자인구수_65_100',
'여자인구수_0 0 14세':'여자인구수_0_14세',
'여자인구수_15 0 64세':'여자인구수_15_64세',
'여자인구수_65 0 100+':'여자인구수_65_100',
'총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
'총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
'총인구수_65 0 100+_비율':'총인구수_65_100_비율',
'남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
'남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
'남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
'여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
'여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
'여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
'10만명당 발생률':'율_10만명당_발생률',
'평균 기온':'평균기온',
'최저 기온':'최저기온',
'최고 기온':'최고기온',
'주말/평일':'주말_평일',
'요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
'요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
'요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
'요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
'연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
'연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
'연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
'연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
2018:'연_2018년',
2019:'연_2019년',
2020:'연_2020년',
2021:'연_2021년',
1:'월_1월',
2:'월_2월',
3:'월_3월',
4:'월_4월',
5:'월_5월',
6:'월_6월',
7:'월_7월',
8:'월_8월',
9:'월_9월',
10:'월_10월',
11:'월_11월',
12:'월_12월',
}, inplace=True)
X_df=전체0df.drop(['배출량'], axis=1)
cols=list(X_df.columns)
단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

selected_vals=[]
for col in cols:
    try:
        res = ols('배출량 ~ '+col, data=전체0df).fit()
        단순회귀df.loc[0, col]=res.rsquared
        if 단순회귀df.loc[0, col]>0.5:
            selected_vals.append(col)
    except:
        print(col)
Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
for val in selected_vals:
    Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
Rsqr_df=Rsqr_df.transpose()
Rsqr_df.rename(columns={0:"Rsqr"}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3831 entries, 0 to 3830
Columns: 346 entries, 일자 to 화
dtypes: float64(244), int64(76), object(1), uint8(25)
memory usage: 9.5+ MB
일자 사업체수 0.9838486383781166
일자 세대수 0.8406097223938047
일자 건설이후(일)_std 0.9490301036111168
일자 고등학교수 0.9918470284159647
일자 거주편차 0.9123718828722309
일자 일편차 0.9068251967474698
일자 방문편차 0.984831373043608
일자 거주표준편차 0.9013643640766998
일자 일표준편차 0.8522555452289111
일자 내국인_합계 0.9364970305359667
일자 외국인_합계 0.9859585366896878
일자 총인구수_계 0.9946896300206477
일자 남자인구수_계 0.971208807327719
일자 여자인구수_계 0.9488804845268834
일자 총인구수_0 0 14세 0.9914497212628021
일자 총인구수_15 0 64세 0.9747447276722949
일자 총인구수_65 0 100+ 0.9472694653741958
일자 남자인구수_0 0 14세 0.9983963196122634
일자 남자인구수_15 0 64세 0.9690583774213315
일자 남자인구수_65 0 100+ 0.9503538976468513
일자 여자인구수_0 0 14세 0.9744129142287987
일자 여자인구수_15 0 64세 0.9713673162456774
일자 일강수량 0.8449971064945075
일자 유흥단란주점 0.9667987963406088
일자 음식점면적합 0.9351231432100946
일자 일 0.9202910602725248
일자 일평균결제건수 0.933330192

건설이후(일) 유흥주점 0.9819964408741719
건설이후(일) 행정동면적 0.9958846525123198
건설이후(일) 면적당일반음식점개수 0.9911858783960066
건설이후(일) 면적당단란주점개수 0.9312374567127012
건설이후(일) 요일마트/슈퍼마켓결제건수비율 0.8216792547235968
건설이후(일) 요일간식결제금액 0.8039452878023371
건설이후(일) 요일양식결제금액 0.8767339473337138
건설이후(일) 요일양식결제건수비율 0.8602826108156023
건설이후(일) 연_월마트/슈퍼마켓결제건수비율 0.8375579069373214
건설이후(일) 연_월결제금액_양식 0.8899209440390144
건설이후(일) 연_월양식결제건수비율 0.8760231129034246
건설이후(일) 연평균1회배출금액 0.9018803758616623
건설이후(일) 월평균배출건수 0.8427681637719221
건설이후(일) 월평균1회배출금액 0.8731382011187556
건설이후(일) 월_일평균배출건수 0.8188479723823282
건설이후(일) 월_일평균1회배출금액 0.970224042240416
건설이후(일) 요일평균배출건수 0.8870203898045688
건설이후(일) 분기 0.9776759396935639
건설이후(일)_std 고등학교수 0.9469456441269459
건설이후(일)_std 거주편차 0.8969838609314795
건설이후(일)_std 일편차 0.9191635096910382
건설이후(일)_std 방문편차 0.9230172809079221
건설이후(일)_std 거주표준편차 0.8892314395559576
건설이후(일)_std 일표준편차 0.9229043315446651
건설이후(일)_std 내국인_합계 0.9992916886300344
건설이후(일)_std 외국인_합계 0.886428159632254
건설이후(일)_std 총인구수_계 0.9160650150338423
건설이후

장기1일표준편차 장기1방문편차 0.9553330026762777
장기1방문표준편차 장기2거주편차 0.9757014646695058
장기2거주표준편차 장기2일편차 0.9565969868914532
장기2일표준편차 장기2방문편차 0.9618513522799402
장기2방문표준편차 장기3거주편차 0.9769857653847656
장기3거주표준편차 장기3일편차 0.9475182509523562
장기3일표준편차 장기3방문편차 0.9543847586420521
장기3방문표준편차 장기4거주편차 0.9932789923113765
장기3방문표준편차 장기4일표준편차 0.8402997445197372
장기3방문표준편차 장기4방문편차 0.8331339170781261
장기3방문표준편차 여자인구수_계_비율 0.8945067330742535
장기3방문표준편차 총인구수_65 0 100+_비율 0.8957966914997556
장기3방문표준편차 남자인구수_65 0 100+_비율 0.8914768838928282
장기3방문표준편차 여자인구수_0 0 14세_비율 0.9157439087263745
장기3방문표준편차 요일간식결제금액 0.8181161124508392
장기3방문표준편차 요일간식결제건수비율 0.8545076416533143
장기3방문표준편차 요일배달결제금액비율 0.8086945813875386
장기3방문표준편차 요일양식결제건수 0.8105782291248348
장기3방문표준편차 요일부페결제금액비율 0.8080443531677942
장기3방문표준편차 연_월결제건수_마트/슈퍼마켓 0.8489541071538997
장기3방문표준편차 연_월패스트푸드결제금액비율 0.8339469483754668
장기3방문표준편차 연_월결제금액_간식 0.8373728786425002
장기3방문표준편차 연_월간식결제건수비율 0.8682565499766774
장기3방문표준편차 연_월배달결제금액비율 0.8865931813606098
장기3방문표준편차 연_월결제건수_양식 0.8934960882246957
장기4거주편

총인구수_15 0 64세 요일평균결제건수 0.962781454994803
총인구수_15 0 64세 요일결제건수편차 0.9451961515915269
총인구수_15 0 64세 요일평균결제금액 0.9215949094637111
총인구수_15 0 64세 요일평균1회결제금액 0.942424066007993
총인구수_15 0 64세 월_일평균결제건수 0.9501949194810639
총인구수_15 0 64세 월_일결제건수편차 0.9257279283154326
총인구수_15 0 64세 월_일평균결제금액 0.8953133370337848
총인구수_15 0 64세 성산읍 0.9494589318770345
총인구수_65 0 100+ 남자인구수_0 0 14세 0.9586249637313401
총인구수_65 0 100+ 남자인구수_15 0 64세 0.9359294172837438
총인구수_65 0 100+ 남자인구수_65 0 100+ 0.9992799607380508
총인구수_65 0 100+ 여자인구수_0 0 14세 0.9937453913279252
총인구수_65 0 100+ 여자인구수_15 0 64세 0.9847772502168781
총인구수_65 0 100+ 여자인구수_계_비율 0.8306255637813427
총인구수_65 0 100+ 총인구수_65 0 100+_비율 0.8328355883599969
총인구수_65 0 100+ 남자인구수_65 0 100+_비율 0.8265352201142387
총인구수_65 0 100+ 유흥단란주점 0.8346937083074096
총인구수_65 0 100+ 일 0.9808642633079803
총인구수_65 0 100+ 일평균결제건수 0.9826595921108167
총인구수_65 0 100+ 일결제건수편차 0.9647399124145012
총인구수_65 0 100+ 일평균결제금액 0.9466553796047675
총인구수_65 0 100+ 일평균1회결제금액 0.9918276954819821
총인구수_65 0 100+ 월평균결제건수 0.

여자인구수_65 0 100+ 연_월간식결제건수비율 0.8786518595652645
여자인구수_65 0 100+ 연_월배달결제금액비율 0.9067912130023732
여자인구수_65 0 100+ 연_월결제건수_양식 0.8876270150807954
여자인구수_65 0 100+ 연_월결제금액_양식 0.9467540076454083
여자인구수_65 0 100+ 연_월양식결제건수비율 0.9544089022934082
여자인구수_65 0 100+ 연평균1회배출금액 0.9068950894561424
여자인구수_65 0 100+ 월평균배출건수 0.9525745423143552
여자인구수_65 0 100+ 월평균1회배출금액 0.8789104382129875
여자인구수_65 0 100+ 월_일평균배출건수 0.9271437734071362
여자인구수_65 0 100+ 월_일평균1회배출금액 0.962802568032702
여자인구수_65 0 100+ 요일평균배출건수 0.9948528613843677
여자인구수_65 0 100+ 분기 0.9642535739820379
인구밀도 남자인구수_0 0 14세_비율 0.9839851613542032
인구밀도 위도 0.9401594278720468
인구밀도 일반음식점 0.9827848670618099
인구밀도 음식점면적평균 0.8476353336342227
남자인구수_계_비율 여자인구수_0 0 14세_비율 0.8268455091128609
남자인구수_계_비율 일강수량월편차 0.987674316658566
남자인구수_계_비율 유흥주점 0.8321299061256368
남자인구수_계_비율 행정동면적 0.9505241164469183
남자인구수_계_비율 면적당일반음식점개수 0.9620128674513787
남자인구수_계_비율 요일간식결제금액 0.9332000329212484
남자인구수_계_비율 요일간식결제건수비율 0.8636761462673953
남자인구수_계_비율 요일배달결제금액비율 0.8003649689541761
남자인구수_계_비율 요일양

...

PM25 월_일평균1회결제금액 1.0
일 일평균결제건수 0.9946509607167102
일 일결제건수편차 0.9872836477948091
일 일평균결제금액 0.9675086940635867
일 일평균1회결제금액 0.9889571518024004
일 월평균결제건수 0.9874119909691556
일 월결제건수편차 0.9872834136364076
일 월평균결제금액 0.9811619457520364
일 월평균1회결제금액 0.9620036994061935
일 연평균결제건수 0.9747495294484148
일 연결제건수편차 0.9604400556790083
일 연평균결제금액 0.9727225126984511
일 연평균1회결제금액 0.9594538038834122
일 연_요일평균결제건수 0.9723886687178783
일 연_요일결제건수편차 0.9504909483830938
일 연_요일평균결제금액 0.9541669683494562
일 요일마트/슈퍼마켓결제금액비율 0.8988654944377626
일 요일한식결제건수 0.8774611871841198
일 요일한식결제금액비율 0.8641388749952518
일 요일식품결제건수 0.8495807894490528
일 요일식품결제금액비율 0.8739505243251404
일 요일패스트푸드결제건수 0.82783788314977
일 요일패스트푸드결제금액비율 0.8499637190770323
일 요일간식결제건수 0.8753097626205935
일 요일간식결제금액비율 0.8818407842962291
일 요일아시아음식결제건수 0.8689420221818173
일 요일아시아음식결제금액비율 0.8038664327023924
일 요일양식결제금액비율 0.8308047088880813
일 요일주점및주류판매결제건수 0.819596781132483
일 요일부페결제금액비율 0.9053925843426118
일 연_월결제건수_마트/슈퍼마켓 0.862876749954652
일 연_월마트/슈퍼마켓결제금액비율 0.9628463936023219


월평균1회결제금액 요일부페결제금액비율 0.9008665816343197
월평균1회결제금액 연_월결제건수_마트/슈퍼마켓 0.8622733739337567
월평균1회결제금액 연_월마트/슈퍼마켓결제금액비율 0.9636148531077654
월평균1회결제금액 연_월결제건수_한식 0.9538633600001252
월평균1회결제금액 연_월한식결제금액비율 0.919132598951944
월평균1회결제금액 연_월결제건수_식품 0.9055320746800989
월평균1회결제금액 연_월식품결제금액비율 0.978644394893319
월평균1회결제금액 연_월결제건수_패스트푸드 0.9626246676510267
월평균1회결제금액 연_월패스트푸드결제금액비율 0.9180946429611699
월평균1회결제금액 연_월결제건수_간식 0.948913905631181
월평균1회결제금액 연_월간식결제금액비율 0.9699988467206654
월평균1회결제금액 연_월결제건수_아시아음식 0.9680166910179082
월평균1회결제금액 연_월아시아음식결제금액비율 0.9175464277483881
월평균1회결제금액 연_월양식결제금액비율 0.8891551387856657
월평균1회결제금액 연_월결제건수_주점및주류판매 0.8845977381938398
월평균1회결제금액 연_월부페결제금액비율 0.9764225344463403
월평균1회결제금액 요일평균결제건수 0.9768311905161435
월평균1회결제금액 요일결제건수편차 0.9681700786933465
월평균1회결제금액 요일평균결제금액 0.9578101909915316
월평균1회결제금액 요일평균1회결제금액 0.9620036994061935
월평균1회결제금액 월_일평균결제건수 0.9616313147452066
월평균1회결제금액 월_일결제건수편차 0.9464438757681466
월평균1회결제금액 월_일평균결제금액 0.9298488394755156
월평균1회결제금액 성산읍 0.8687403921523673
연평균결제건수 연결제건수편차 0.990291

연_월결제금액비율 연_요일별결제건수비율 0.8825107775858333
연_요일별결제금액비율 요일마트/슈퍼마켓결제건수 0.9764584625241761
연_요일별결제금액비율 요일마트/슈퍼마켓결제금액비율 0.857070229334296
연_요일별결제금액비율 요일한식결제건수 0.8618459914160951
연_요일별결제금액비율 요일식품결제금액비율 0.9118238352718823
연_요일별결제금액비율 요일패스트푸드결제건수 0.9139706913990779
연_요일별결제금액비율 요일패스트푸드결제금액비율 0.9652831465102167
연_요일별결제금액비율 요일간식결제건수 0.9551555867811435
연_요일별결제금액비율 요일간식결제금액비율 0.8300472503004975
연_요일별결제금액비율 요일아시아음식결제건수 0.8666159806797658
연_요일별결제금액비율 요일아시아음식결제금액비율 0.8988691553568121
연_요일별결제금액비율 요일부페결제금액비율 0.8073854861747776
연_요일별결제금액비율 연_월패스트푸드결제금액비율 0.8034654449445754
요일마트/슈퍼마켓결제건수 요일한식결제건수 0.8105906343688823
요일마트/슈퍼마켓결제건수 요일식품결제금액비율 0.8901327508405618
요일마트/슈퍼마켓결제건수 요일패스트푸드결제건수 0.9078171179061978
요일마트/슈퍼마켓결제건수 요일패스트푸드결제금액비율 0.9337011000389642
요일마트/슈퍼마켓결제건수 요일간식결제건수 0.92205613095509
요일마트/슈퍼마켓결제건수 요일아시아음식결제건수 0.8429305132636143
요일마트/슈퍼마켓결제건수 요일아시아음식결제금액비율 0.840213826178932
요일마트/슈퍼마켓결제건수 요일양식결제건수 0.8189826922200144
요일마트/슈퍼마켓결제건수 요일부페결제금액비율 0.8042783351594333
요일마트/슈퍼마켓결제건수 연_월결제건수_마트/슈퍼마켓 0.8014604919760

요일배달결제금액 연_월배달결제건수비율 0.9378981149254121
요일배달결제금액 2020 0.8396673537489808
요일배달결제건수비율 연_월농축수산물결제금액비율 0.8268958837813091
요일배달결제건수비율 연_월결제건수_배달 0.8317981780328412
요일배달결제건수비율 연_월결제금액_배달 0.9343183483842007
요일배달결제건수비율 연_월배달결제건수비율 0.9403018371888018
요일배달결제건수비율 2020 0.8389818349293159
요일배달결제금액비율 요일양식결제건수 0.9876752446407664
요일배달결제금액비율 요일양식결제금액 0.8871574287916897
요일배달결제금액비율 요일양식결제건수비율 0.8956111182620765
요일배달결제금액비율 연_월결제금액_간식 0.8511327584729729
요일배달결제금액비율 연_월간식결제건수비율 0.8244093716413539
요일배달결제금액비율 연_월배달결제금액비율 0.8457809338449104
요일배달결제금액비율 연_월결제건수_양식 0.8405044906817033
요일배달결제금액비율 연_월양식결제건수비율 0.8142749802872314
요일배달결제금액비율 월평균배출건수 0.8220960864921631
요일배달결제금액비율 월_일평균배출건수 0.8006554073285644
요일배달결제금액비율 요일평균배출건수 0.8316734933180139
요일양식결제건수 요일양식결제금액 0.8441611383576374
요일양식결제건수 요일양식결제건수비율 0.8678779435014554
요일양식결제건수 연_월결제금액_간식 0.8284879375841377
요일양식결제건수 연_월간식결제건수비율 0.8120598565028748
요일양식결제건수 연_월배달결제금액비율 0.8327238439361488
요일양식결제건수 연_월결제건수_양식 0.8324510296895347
요일양식결제건수 요일평균배출건수 0.8018019113950281
요일양식결제금액

연_월배달결제금액비율 연_월양식결제건수비율 0.9291777750631328
연_월배달결제금액비율 연평균1회배출금액 0.8010867820708719
연_월배달결제금액비율 월평균배출건수 0.910356626800867
연_월배달결제금액비율 월_일평균배출건수 0.8873084995544126
연_월배달결제금액비율 월_일평균1회배출금액 0.8077795746059635
연_월배달결제금액비율 요일평균배출건수 0.9044067262179509
연_월배달결제금액비율 분기 0.8054824309356285
연_월결제건수_양식 연_월결제금액_양식 0.876416834806686
연_월결제건수_양식 연_월양식결제건수비율 0.9070060541739076
연_월결제건수_양식 월평균배출건수 0.8867567408768335
연_월결제건수_양식 월_일평균배출건수 0.8643413023417742
연_월결제건수_양식 요일평균배출건수 0.8844132366429044
연_월결제금액_양식 연_월양식결제건수비율 0.991353211458818
연_월결제금액_양식 연평균1회배출금액 0.8935728293737871
연_월결제금액_양식 월평균배출건수 0.9289354524200729
연_월결제금액_양식 월평균1회배출금액 0.8657658967878428
연_월결제금액_양식 월_일평균배출건수 0.904111387329169
연_월결제금액_양식 월_일평균1회배출금액 0.9415939616742076
연_월결제금액_양식 요일평균배출건수 0.9496190503201527
연_월결제금액_양식 분기 0.9443064870910113
연_월양식결제건수비율 연평균1회배출금액 0.8910038124067516
연_월양식결제건수비율 월평균배출건수 0.9325492414463241
연_월양식결제건수비율 월평균1회배출금액 0.8632901971280059
연_월양식결제건수비율 월_일평균배출건수 0.907724262506288
연_월양식결제건수비율 월_일평균1회배출금액 0.9375114039992948
연_월양식

In [108]:
Rsqr_df

,Rsqr
건설이후,0.625671
요일부페결제건수,0.509427


## 행정동별 Rsqr 찾기

In [109]:
행정동명1=["건입동", "남원읍", "노형동", "대륜동", "대정읍", "대천동", "도두동", "동홍동", 
      "봉개동", "삼도1동", "삼도2동", "삼양동", "서홍동", "성산읍", "송산동", "아라동", "안덕면",
      "애월읍", "연동", "영천동", "예래동", "오라동", "외도동", "용담1동", "용담2동", 
      "이도1동", "이도2동", "이호동", "일도1동", "일도2동", "정방동", "중문동", "중앙동", 
      "천지동", "표선면", "화북동", "효돈동"]

In [113]:
Rsqr_all=pd.DataFrame([[0]*len(행정동명1)], columns=행정동명1)

In [125]:
for 행정동 in 행정동명1:
    # 설명 변수
    행0_df=pd.DataFrame([[0]*len(df1.iloc[0, :])], columns=list(df1.columns))
    행0_df.drop([0], inplace=True)
    dfdf=df1[df1["행정동명"]==행정동]
    행0_df=pd.concat([행0_df, dfdf])
    행0_df=행0_df.sort_values(['일자', '행정동명'])
    행0_df=행0_df.reset_index()
    행0_df.drop(['index'], axis=1, inplace=True)

    # 반응 변수
    행1표적_df=pd.read_csv("../전처리/평가데이터(처리)1.csv", encoding="cp949")
    배출량=pd.DataFrame([[0]*len(행1표적_df.iloc[0, :])], columns=list(행1표적_df.columns))
    배출량.drop([0], inplace=True)

    dfdf=행1표적_df[행1표적_df["행정동명"]==행정동]
    배출량=pd.concat([배출량, dfdf])
    배출량=배출량.sort_values(['일자', '행정동명'])
    배출량=배출량.reset_index()
    배출량.drop(['index'], axis=1, inplace=True)

    #행0_df

    #배출량

    행정동원핫1=pd.get_dummies(행0_df['행정동명'])
    행정동원핫2=pd.get_dummies(행0_df['년'])
    행정동원핫3=pd.get_dummies(행0_df['월'])
    행정동원핫4=pd.get_dummies(행0_df['요일'])

    행0_df=pd.concat([행0_df, 행정동원핫1, 행정동원핫2, 행정동원핫3, 행정동원핫4], axis=1)
    행0_df.drop(['행정시','행정동명', '년', '월', '요일', 'Unnamed: 0'], axis=1, inplace=True)
    #행0_df.info()

    y=배출량['배출량']## 반응 변수
    전체0df=pd.concat([행0_df, y], axis=1)
    #전체0df

    """
    cols=list(전체0df.columns)
    corr_1=전체0df.corr()
    for i in range(len(corr_1)):
        for j in range(i+1, len(corr_1.iloc[0, :])):
            if corr_1.iloc[i,j]>0.8:
                print(cols[i], cols[j], corr_1.iloc[i,j])
    """

    전체0df.drop(['일자', '일',  '일.1'], axis=1, inplace=True)
    전체0df.rename(columns={'세대당 인구':"세대당인구", "남여 비율":"남여비율", "동.식물관련시설":"동식물", 
    "분뇨.쓰레기처리시설":'',
    '건설이후(일)':'건설이후',
    '건설이후(일)_std':'건설이후_std',
    '내국인_행태별 0 개별여행':'내국인_행태별_개별여행',
    '내국인_행태별 0 부분패키지':'내국인_행태별_부분패키지',
    '내국인_행태별 0 패키지':'내국인_행태별_패키지',
    '내국인_목적별 0 레저스포츠':'내국인_목적별_레저스포츠',
    '내국인_목적별 0 회의및업무':'내국인_목적별_회의및업무',
    '내국인_목적별 0 휴양및관람':'내국인_목적별_휴양및관람',
    '내국인_목적별 0 친지방문':'내국인_목적별_친지방문',
    '내국인_목적별 0 교육여행':'내국인_목적별_교육여행',
    '내국인_목적별 0 기타방문':'내국인_목적별_기타방문',
    '총인구수_0 0 14세':'총인구수_0_14세',
    '총인구수_15 0 64세':'총인구수_15_64세',
    '총인구수_65 0 100+':'총인구수_65_100',
    '남자인구수_0 0 14세':'남자인구수_0_14세',
    '남자인구수_15 0 64세':'남자인구수_15_64세',
    '남자인구수_65 0 100+':'남자인구수_65_100',
    '여자인구수_0 0 14세':'여자인구수_0_14세',
    '여자인구수_15 0 64세':'여자인구수_15_64세',
    '여자인구수_65 0 100+':'여자인구수_65_100',
    '총인구수_0 0 14세_비율':'총인구수_0_14세_비율',
    '총인구수_15 0 64세_비율':'총인구수_15_64세_비율',
    '총인구수_65 0 100+_비율':'총인구수_65_100_비율',
    '남자인구수_0 0 14세_비율':'남자인구수_0_14세_비율',
    '남자인구수_15 0 64세_비율':'남자인구수_15_64세_비율',
    '남자인구수_65 0 100+_비율':'남자인구수_65_100_비율',
    '여자인구수_0 0 14세_비율':'여자인구수_0_14세_비율',
    '여자인구수_15 0 64세_비율':'여자인구수_15_64세_비율',
    '여자인구수_65 0 100+_비율':'여자인구수_65_100_비율',
    '10만명당 발생률':'율_10만명당_발생률',
    '평균 기온':'평균기온',
    '최저 기온':'최저기온',
    '최고 기온':'최고기온',
    '주말/평일':'주말_평일',
    '요일마트/슈퍼마켓결제건수':'요일마트_슈퍼마켓결제건수',
    '요일마트/슈퍼마켓결제금액':'요일마트_슈퍼마켓결제금액',
    '요일마트/슈퍼마켓결제건수비율':'요일마트_슈퍼마켓결제건수비율',
    '요일마트/슈퍼마켓결제금액비율':'요일마트_슈퍼마켓결제금액비율',
    '연_월결제건수_마트/슈퍼마켓':'연_월결제건수_마트_슈퍼마켓',
    '연_월결제금액_마트/슈퍼마켓':'연_월결제금액_마트_슈퍼마켓',
    '연_월마트/슈퍼마켓결제건수비율':'연_월마트_슈퍼마켓결제건수비율',
    '연_월마트/슈퍼마켓결제금액비율':'연_월마트_슈퍼마켓결제금액비율',
    2018:'연_2018년',
    2019:'연_2019년',
    2020:'연_2020년',
    2021:'연_2021년',
    1:'월_1월',
    2:'월_2월',
    3:'월_3월',
    4:'월_4월',
    5:'월_5월',
    6:'월_6월',
    7:'월_7월',
    8:'월_8월',
    9:'월_9월',
    10:'월_10월',
    11:'월_11월',
    12:'월_12월',
    }, inplace=True)
    X_df=전체0df.drop(['배출량'], axis=1)
    cols=list(X_df.columns)
    단순회귀df=pd.DataFrame([[0]*len(cols)], columns=cols)

    selected_vals=[]
    for col in cols:
        try:
            res = ols('배출량 ~ '+col, data=전체0df).fit()
            단순회귀df.loc[0, col]=res.rsquared
            if 단순회귀df.loc[0, col]>0.5:
                selected_vals.append(col)
        except:
            print(col)
    Rsqr_df=pd.DataFrame([[0]*len(selected_vals)], columns=selected_vals)
    for val in selected_vals:
        Rsqr_df.loc[0, val]=단순회귀df.loc[0, val]
    Rsqr_df=Rsqr_df.transpose()
    print(행정동)
    print(Rsqr_df)
    #Rsqr_df.rename(columns={0:행정동}, inplace=True)
    #Rsqr_all=pd.concat([Rsqr_all, Rsqr_df], axis=1)
    


건입동
Empty DataFrame
Columns: [0]
Index: []

남원읍
Empty DataFrame
Columns: [0]
Index: []

노형동
Empty DataFrame
Columns: [0]
Index: []

대륜동
Empty DataFrame
Columns: [0]
Index: []

대정읍
                        0
세대수              0.559417
세대당인구            0.675002
건설이후             0.672382
총인구수_65_100      0.577537
남자인구수_65_100     0.563852
여자인구수_65_100     0.580138
남자인구수_계_비율       0.587990
여자인구수_계_비율       0.587990
총인구수_65_100_비율   0.626231
남자인구수_65_100_비율  0.609567
여자인구수_65_100_비율  0.610840
연평균결제건수          0.682101
연결제건수편차          0.686484
연평균결제금액          0.633074
요일배달결제건수         0.545259
요일배달결제금액         0.538900
요일부페결제건수비율       0.525092
연배출건수편차          0.704162
연평균1회배출금액        0.712482
연_요일배출건수편차       0.698559
연_요일평균1회배출금액     0.690519
연_2018년          0.620173

대천동
Empty DataFrame
Columns: [0]
Index: []

도두동
                  0
월_일평균배출량   0.532603
월_일평균배출금액  0.532662

동홍동
Empty DataFrame
Columns: [0]
Index: []

봉개동
Empty DataFrame
Columns: [0]
Index: []

삼도1동
Empty DataFrame
Co